# BUILDING A SPAM FILTER

We are going to design a spam filter using the Naive Bayes algorithm, that will essentially "teach" our computer how to classify messages. In order to classify a message as spam or non-spam, we need the computer to: 

-Learns how humans classify messages.
-Uses that human knowledge to estimate probabilities for new messages — -probabilities for spam and non-spam.
-Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

To teach the computer how to classify messages, we'll be using the Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

(dataset can be downloaded from this link:https://dq-content.s3.amazonaws.com/433/SMSSpamCollection)

In [1]:
import pandas as pd
data_df = pd.read_csv('SMSSpamCollection', sep= '\t', header = None, names = ['Label', 'SMS'])
print(data_df.shape)


(5572, 2)


In [2]:
data_df.head()


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data_df['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## TRAINING AND TESTING SETS

About 87% of the dataset falls under the non-spam or 'ham' category and 13% were considered spam. Before creating the filter, we will need to think of a way to test how well it works. To avoid a bias testing pool, we will design the test prior to developing the software.

Once the spam filter is done, we need to test how good it is with classifying new messages. To test the spam filter, we are first going to split our dataset into two categories:
(1)A training set, which we will us to "train" the computer how to classify messages
(2) A test set, which we will use to test how good the spam filter is with classifying new messages

The training set will have 4,458(about 80% of the dataset) and the test set will have 1,114 messages(about 20% of the dataset).

Our goal is to create a spam filter that will filter the new messages with an accuracy of at least 80%. We will compare these to the correct answers in the test set to determine the accuracy of the spam filter.



In [4]:
data_random = data_df.sample(frac=1, random_state=1)
training_set_index = round(len(data_random)*0.8)

training_set = data_random[:training_set_index].reset_index(drop=True)
testing_set = data_random[training_set_index:].reset_index(drop=True)

print(training_set.shape)
print(testing_set.shape)

(4458, 2)
(1114, 2)


In [5]:
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:
testing_set.head()

,Label,SMS
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."


In [7]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [8]:
testing_set['Label'].value_counts(normalize = True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The data has been randomized and assigned into the sets with our predetermined percentage in each and the results look similar to the full dataset in both sets.

## DATA CLEANING

In order to calculate all the probabilities with the Naive Bayes algorithm, we will need to clean the data in order to get accurate percentages of each occurrence of a word. For example, an occurrence of the word 'SECRET' would be counted separate from the word 'secret' due to the case sensitivity of the programming language. By cleaning the data and removing punctuation, we can avoid this and create a more accurate program.

In [9]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


We have now removed all the punctuation from the training_set and changed all the letters to lowercase. With the exception of the 'Label' column, every other column in the transformed dataset represents a unique word in our vocabulary. We will need to create a list with all of the unique words that occur in the messages of our training set.

In [10]:
training_set['SMS'] = training_set['SMS'].str.split()
vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))
len(vocabulary)
    

7783

## THE FINAL TRAINING SET

There seems to be 7,783 unique words in our vocabulary list. We will now use the vocabulary to make the data transformation we need. To do this, we'll need to first build a dictionary that we will eventually convert into the DataFrame we need.

In [11]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

word_counts= pd.DataFrame(word_counts_per_sms)
word_counts.head()


,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [12]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## CALCULATING CONSTANTS

Now that our data is clean and organized, we can begin calculating probabilities. Some of the terms in the Naive Bayes algorithm will have the same value for every new message:
P(Spam) and P(Ham): the probabilities of messages that are spam or ham
N(spam), N(ham), N(vocabulary): the number of words in all spam messages, number of words in all non-spam messages, messages, and number of words in our vocabulary. We will also be using Laplace smoothing and set it equal to 1.

In [13]:
alpha = 1
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)


n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()


n_vocabulary = len(vocabulary)

## CALCULATING PARAMETERS

We have calculated all the constants we will need in our algorithm for every new message. The probability of a word given the message is 'ham' will differ from the probability of the same word given the message is 'spam'. We'll need to calculate these probabilities and we can take identical steps to calculate them.

We have 7,783 words in our vocabulary, meaning we will need to calculate a total of 15,566 probabilities(2 for each word: one for spam message, one for ham message)

In [14]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## CLASSIFYING A NEW MESSAGE

Now that we've calculated all the constants and parameters we need, we can create the spam filter. The spam filter can be understood as a function that:

-Takes in as input a new message
-Calculates P(Spam) and P(Ham) given the words in the message
-Compares the values of these conditional probabilities
-Assigns the message as Spam or Ham based on which probability is greater
-If probabilites are equal, it may request human help.

In [15]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [16]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [17]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## MEASURING THE SPAM FILTER'S ACCURACY

We managed to create a spam filter and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages. The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label(given by a human).

In [20]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
testing_set['predicted'] = testing_set['SMS'].apply(classify_test_set)
testing_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [22]:
correct = 0
total = testing_set.shape[0]

for row in testing_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct +=1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


The accuracy is close to 98.7%, which is extremely good. Our spam filter looked at the 1,114 messages that it hadn't seen in training, and classified 1,100 correctly

## NEXT STEPS

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

Next steps include:

Analyze the 14 messages that were classified incorrectly and try to figure out why the algorithm classified them incorrectly
Make the filtering process more complex by making the algorithm sensitive to letter case